<a href="https://colab.research.google.com/github/Khaled2049/CSCI-5832/blob/main/NLP_shared_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [59]:
import requests
import numpy as np
import spacy
import bs4
import urllib3
import json

In [60]:
import torch
from torchcrf import CRF as CRFDecoder
from typing import List, Tuple

In [3]:
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Train Data Exploration

In [ ]:
split_data = requests.get('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json', allow_redirects=True)
train = split_data.json()
dev_data = requests.get('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json', allow_redirects=True)
# is this supposed to be dev_data.json() ?
dev = split_data.json()

<Response [200]>


In [ ]:
train[0]['annotations'][0]['result']
annotations = pd.DataFrame.from_dict(pd.json_normalize(train[0]['annotations'][0]['result']), orient='columns')
annotations

,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0,116,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,8d41599e98424d9480c25109556a7d14,labels,text,label,116,678,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678,964,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,4825806388fe43d39f73354b10b5b32d,labels,text,label,964,1093,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093,1180,\n Heard the learned Counsel for the app...,[NONE]
...,...,...,...,...,...,...,...,...
86,375bebe6234540bf92865406f84bece0,labels,text,label,16196,16241,The judgment of the court below is set aside.,[RPC]
87,70cb6f740b1746259612a99bc2df8062,labels,text,label,16242,16269,The appellant is acquitted.,[RPC]
88,9cdcf8bd45584c7097d1444920690609,labels,text,label,16270,16335,"The fine amount, if any, paid by the appellant...",[RPC]
89,9d3cd3d2675b4f78a936b4b3256b8862,labels,text,label,16335,16524,\n The office is directed to communicate this ...,[RPC]


In [ ]:
train_data = pd.DataFrame.from_dict(pd.json_normalize(train), orient='columns')
train_data.drop('annotations', axis=1, inplace=True)
train_data.drop('id', axis=1, inplace=True)
train_data.head()

,data.text,meta.group
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal
1,...,Tax
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax


In [ ]:
X = pd.concat([train_data, annotations], axis = 1)
X.head()

,data.text,meta.group,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0.0,116.0,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,...,Tax,8d41599e98424d9480c25109556a7d14,labels,text,label,116.0,678.0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678.0,964.0,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax,4825806388fe43d39f73354b10b5b32d,labels,text,label,964.0,1093.0,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093.0,1180.0,\n Heard the learned Counsel for the app...,[NONE]


In [ ]:
labset = []
for lab in X["value.labels"]:
  if lab not in labset:
    labset.append(lab)
print(labset)

[['PREAMBLE'], ['NONE'], ['FAC'], ['ARG_RESPONDENT'], ['RLC'], ['ARG_PETITIONER'], ['ANALYSIS'], ['PRE_RELIED'], ['RATIO'], ['RPC'], nan]


# Dev Data Exploration

In [ ]:
dev[0]['annotations'][0]['result']
annotations = pd.DataFrame.from_dict(pd.json_normalize(dev[0]['annotations'][0]['result']), orient='columns')
annotations

,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0,116,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,8d41599e98424d9480c25109556a7d14,labels,text,label,116,678,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678,964,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,4825806388fe43d39f73354b10b5b32d,labels,text,label,964,1093,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093,1180,\n Heard the learned Counsel for the app...,[NONE]
...,...,...,...,...,...,...,...,...
86,375bebe6234540bf92865406f84bece0,labels,text,label,16196,16241,The judgment of the court below is set aside.,[RPC]
87,70cb6f740b1746259612a99bc2df8062,labels,text,label,16242,16269,The appellant is acquitted.,[RPC]
88,9cdcf8bd45584c7097d1444920690609,labels,text,label,16270,16335,"The fine amount, if any, paid by the appellant...",[RPC]
89,9d3cd3d2675b4f78a936b4b3256b8862,labels,text,label,16335,16524,\n The office is directed to communicate this ...,[RPC]


In [ ]:
dev_data = pd.DataFrame.from_dict(pd.json_normalize(dev), orient='columns')
dev_data.drop('annotations', axis=1, inplace=True)
dev_data.drop('id', axis=1, inplace=True)
dev_data.head()

,data.text,meta.group
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal
1,...,Tax
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax


In [ ]:
X_dev = pd.concat([dev_data, annotations], axis = 1)
X_dev.head()

,data.text,meta.group,id,type,to_name,from_name,value.start,value.end,value.text,value.labels
0,"IN THE HIGH COURT OF KARNATAKA,\n ...",Criminal,d7a902fe9c23417499a7ef782f9fbdeb,labels,text,label,0.0,116.0,"IN THE HIGH COURT OF KARNATAKA,\n ...",[PREAMBLE]
1,...,Tax,8d41599e98424d9480c25109556a7d14,labels,text,label,116.0,678.0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,[PREAMBLE]
2,IN THE HIGH COURT OF KARNATAKA ...,Criminal,e501424117da40a7935c2d9f2fb2fe38,labels,text,label,678.0,964.0,This Criminal Appeal is filed under Section 37...,[PREAMBLE]
3,PETITIONER:\nRAGHUBAR MANDAL HARIHAR MANDAL\n\...,Tax,4825806388fe43d39f73354b10b5b32d,labels,text,label,964.0,1093.0,\n\n This appeal coming on for hearing t...,[PREAMBLE]
4,PETITIONER:\nP.K. BADIANI\n\n Vs.\n\nRESP...,Tax,d6893a25f82948f8be17fc9e876fb716,labels,text,label,1093.0,1180.0,\n Heard the learned Counsel for the app...,[NONE]


In [ ]:
text = dev[0]['data']['text']
words = nltk.word_tokenize(text)

In [ ]:
pos_tags = nltk.pos_tag(words)

In [ ]:
pos_tags

In [ ]:
chunks = nltk.ne_chunk(pos_tags, binary=True)
#for chunk in chunks:
# print(chunk)

In [ ]:
entities = []
labels = []

for chunk in chunks:
  if hasattr(chunk, 'label'):
    entities.append(' '.join(c[0] for c in chunk))
    labels.append(chunk.label())

entites_labels = list(set(zip(entities, labels)))
entites_df = pd.DataFrame(entites_labels)
entites_df.columns = ["Entities", "labels"]
entites_df

,Entities,labels
0,Laxmibai,NE
1,Central Jail,NE
2,CIRCUIT,NE
3,Chhattisgarh,NE
4,Shri Ishwar Raj,NE
5,Taluk Chincholi,NE
6,GULBARGA,NE
7,Bidar District,NE
8,Bhagewati,NE
9,IPC,NE


# Training Dataset Creation

In [29]:
df = pd.read_json("./drive/MyDrive/NER_TRAIN_JUDGEMENT.json")

In [30]:
datas = {}
count = 0
for info in df['data']:
  datas.update({count:info.get('text')})
  count+=1

In [62]:
metas = {}
count = 0
for info in df['meta']:
  metas.update({count:info.get('source').split("_")[0]})
  count+=1

In [32]:
ids = {}
count = 0
for info in df['id']:
  ids.update({count:info})
  count+=1

In [63]:
# load data using Python JSON module
with open('./drive/MyDrive/NER_TRAIN_JUDGEMENT.json','r') as f:
    judge = json.loads(f.read())
    
# Normalizing data
annotations = pd.json_normalize(judge, record_path =['annotations'])

In [64]:
results = {}
count = 0
for entry in annotations['result']:
  if entry != []:
    for val in entry:
      results.update({str(count)+","+val.get('value').get('text'):val.get('value').get('labels')})
  count+=1

In [65]:
com = pd.DataFrame()

i_col = range(len(ids.values()))
com['index'] = i_col

id_col = ids.values()
com['id'] = id_col

data_col = datas.values()
com['data'] = data_col

meta_col = metas.values()
com['meta'] = meta_col

com

,index,id,data,meta
0,0,90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,1,a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
2,2,33cface31108441080976035ee04d07e,"\n5.2 CW3 Mr Vijay Mishra , Deputy Manager, H...",financial
3,3,08debe900b51464e9f26b5d1eecae170,You are hereby asked not to carry out any cons...,civil
4,4,428070222cd940eba78b233829eb28b8,The pillion rider T.V. Satyanarayana Murthy al...,motorvehicles
...,...,...,...,...
9430,9430,5cb430a6d0e14e57a968f0c43d219682,It is prayed in the application that the suit ...,civil
9431,9431,6e9147c9e6344865a8b095fb7f43f7c2,"In the first instance, Mr.A.D.Desai contended ...",tax
9432,9432,657ef5df817d4e95a36d702b4e41ce7d,"Sikri, J. (as he then was), speaking for the m...",constitution
9433,9433,9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour


In [36]:
train = pd.DataFrame()

texts = []
labs = []
cids = []
for entry in results:
  labs.append(results.get(entry))
  cid = entry.split(",")[0]
  text = entry.split(",")[1]
  texts.append(text)
  cids.append(cid)
  count+=1

train['text'] = texts
train['label'] = labs
train['case id'] = cids

train

,text,label,case id
0,Hongkong Bank,[ORG],0
1,Rahul & Co.,[ORG],0
2,Agya,[OTHER_PERSON],1
3,Kaur,[OTHER_PERSON],1
4,Tarlochan Singh,[OTHER_PERSON],1
...,...,...,...
16992,RAJENDRA MAHAJAN,[JUDGE],9433
16993,M.Cr.C. No. 8763/2009,[CASE_NUMBER],9433
16994,04/02/2015,[DATE],9433
16995,Wakeel Khan,[OTHER_PERSON],9433


In [37]:
full = []
fmeta = []
fid = []
for idx, entity in train.iterrows():
  num = int(entity["case id"])
  full.append(com.at[num,"data"])
  fmeta.append(com.at[num,"meta"])
  fid.append(com.at[num,"id"])

train["case"] = full
train["meta"] = fmeta
train["case id"] = fid

train

,text,label,case id,case,meta
0,Hongkong Bank,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,Rahul & Co.,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
2,Agya,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
3,Kaur,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
4,Tarlochan Singh,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal
...,...,...,...,...,...
16992,RAJENDRA MAHAJAN,[JUDGE],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16993,M.Cr.C. No. 8763/2009,[CASE_NUMBER],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16994,04/02/2015,[DATE],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour
16995,Wakeel Khan,[OTHER_PERSON],9ff4b03ee09f49a6814b9550c145ea1e,(RAJENDRA MAHAJAN) JUDGE AKM M.Cr.C. No. 8763/...,industrial&labour


In [39]:
df2 = pd.read_json("./drive/MyDrive/NER_TRAIN_PREAMBLE.json")

In [40]:
datas2 = {}
count = 0
for info in df2['data']:
  datas2.update({count:info.get('text')})
  count+=1

In [41]:
metas2 = {}
count = 0
for info in df2['meta']:
  metas2.update({count:info.get('source').split("_")[0]})
  count+=1

In [42]:
ids2 = {}
count = 0
for info in df2['id']:
  ids2.update({count:info})
  count+=1

In [43]:
# load data using Python JSON module
with open('./drive/MyDrive/NER_TRAIN_PREAMBLE.json','r') as f:
  pre2 = json.loads(f.read())
    
# Normalizing data
annotations2 = pd.json_normalize(pre2, record_path =['annotations'])

In [44]:
results2 = {}
count = 0
for entry in annotations2['result']:
  if entry != []:
    for val in entry:
      results2.update({str(count)+","+val.get('value').get('text'):val.get('value').get('labels')})
  count+=1

In [45]:
com2 = pd.DataFrame()

i_col2 = range(len(ids2.values()))
com2['index'] = i_col2

id_col2 = ids2.values()
com2['id'] = id_col2

data_col2 = datas2.values()
com2['data'] = data_col2

meta_col2 = metas2.values()
com2['meta'] = meta_col2

com2

,index,id,data,meta
0,0,d79fb7f965a74e418212458285c7c213,In The High Court Of Kerala At Ernakulam\n\nCr...,criminal
1,1,10d82191b30e4e9d9b0ab91c34606631,In The Court Of Shri Lokesh Kumar Sharma\n ...,constitution
2,2,b150ace41ceb4acf87d12f75b4748ffc,Before The Madurai Bench Of Madras High Court\...,civil
3,3,278f93b03a484705b3d96a72aa09be26,Before The Madurai Bench Of Madras High Court\...,tax
4,4,ceab6a1b200242208a4d6beb042d8743,1 ...,criminal
...,...,...,...,...
1555,1555,2b617380eac44e85a80d53e9c067360b,In The High Court Of Judicature At Madras ...,motorvehicles
1556,1556,799133f87be04599a0b5367a4a1a23bf,Petitioner:\nKhudiram Das\n\n\tVs.\n\nResponde...,criminal
1557,1557,d21055fa778e4579a5711c6ffb1b9560,In The High Court Of Judicature At Patna\n\n ...,constitution
1558,1558,307722b0639841c9b31418fbc84d7a52,Petitioner:\nThe Automobile Transport(Rajastha...,tax


In [46]:
train2 = pd.DataFrame()

texts2 = []
labs2 = []
cids2 = []
for entry in results2:
  labs2.append(results2.get(entry))
  cid = entry.split(",")[0]
  text = entry.split(",")[1]
  texts2.append(text)
  cids2.append(cid)
  count+=1

train2['text'] = texts2
train2['label'] = labs2
train2['case id'] = cids2

train2

,text,label,case id
0,High Court Of Kerala At Ernakulam,[COURT],0
1,T.R.Ajayan,[PETITIONER],0
2,M.Ravindran,[RESPONDENT],0
3,Nirmala Dinesh,[RESPONDENT],0
4,A.Kumar,[LAWYER],0
...,...,...,...
11257,V. Nath,[JUDGE],1557
11258,Automobile Transport(Rajasthan) Ltd.,[PETITIONER],1558
11259,State Of Rajasthan,[RESPONDENT],1558
11260,East India COMMERClAL Co.,[PETITIONER],1559


In [47]:
full2 = []
fmeta2 = []
fid2 = []
for idx, entity in train2.iterrows():
  num = int(entity["case id"])
  full2.append(com.at[num,"data"])
  fmeta2.append(com.at[num,"meta"])
  fid2.append(com.at[num,"id"])

train2["case"] = full2
train2["meta"] = fmeta2
train2["case id"] = fid2

train2

,text,label,case id,case,meta
0,High Court Of Kerala At Ernakulam,[COURT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
1,T.R.Ajayan,[PETITIONER],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
2,M.Ravindran,[RESPONDENT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
3,Nirmala Dinesh,[RESPONDENT],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
4,A.Kumar,[LAWYER],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax
...,...,...,...,...,...
11257,V. Nath,[JUDGE],dd0bd3b3088541909bf3c88481254dd4,Lord Dunedin expressed the view that there mig...,industrial&labour
11258,Automobile Transport(Rajasthan) Ltd.,[PETITIONER],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil
11259,State Of Rajasthan,[RESPONDENT],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil
11260,East India COMMERClAL Co.,[PETITIONER],71671326f0f64d8da7f5f4c5a08ce156,It is pertinent to mention here that from the ...,criminal


In [48]:
cat_col = []
for i in range(len(train)):
  cat_col.append("JUDGEMENT")
train["category"] = cat_col

cat_col2 = []
for i in range(len(train2)):
  cat_col2.append("PREAMBLE")
train2["category"] = cat_col2

In [54]:
train_data = pd.concat([train, train2], ignore_index=True)
train_data

,text,label,case id,case,meta,category
0,Hongkong Bank,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax,JUDGEMENT
1,Rahul & Co.,[ORG],90d9a97c7b7749ec8a4f460fda6f937e,\n\n(7) On specific query by the Bench about a...,tax,JUDGEMENT
2,Agya,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal,JUDGEMENT
3,Kaur,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal,JUDGEMENT
4,Tarlochan Singh,[OTHER_PERSON],a325c57ba5b84c6fa46bee65e6616633,"He was also asked whether Agya <span class=""hi...",criminal,JUDGEMENT
...,...,...,...,...,...,...
28254,V. Nath,[JUDGE],dd0bd3b3088541909bf3c88481254dd4,Lord Dunedin expressed the view that there mig...,industrial&labour,PREAMBLE
28255,Automobile Transport(Rajasthan) Ltd.,[PETITIONER],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil,PREAMBLE
28256,State Of Rajasthan,[RESPONDENT],91efcd1b8a46467c8e772f736ae4b41d,Ugamchand D.W. 14 also corroborates Inderchand...,civil,PREAMBLE
28257,East India COMMERClAL Co.,[PETITIONER],71671326f0f64d8da7f5f4c5a08ce156,It is pertinent to mention here that from the ...,criminal,PREAMBLE


# Model Setup (under construction)

In [55]:
ner_labs = []
for lab in train_data['label']:
  if not lab in ner_labs:
    ner_labs.append(lab)

print(ner_labs)

[['ORG'], ['OTHER_PERSON'], ['WITNESS'], ['GPE'], ['STATUTE'], ['DATE'], ['PROVISION'], ['COURT'], ['PRECEDENT'], ['CASE_NUMBER'], ['PETITIONER'], ['JUDGE'], ['RESPONDENT'], ['LAWYER']]


In [57]:
# For feature encodings. We have a special UNKNOWN feature
UNK_SYMBOL = "<UNK>"
# For padding our variable length batches
PAD_SYMBOL = "<PAD>"


def load_data(
    filepath: str,
) -> Tuple[List[str], List[str]]:
    """Load the training data, producing a List of sentences, each comprising
    a List of tokens, and a List of their corresponding NER tags

    Args:
        filepath (str): Path to the training file

    Returns:
       Tuple[List[str], List[str]]: The tokens and tags
    """
    token_sents = []
    tag_sents = []
    token_sent = []
    tag_sent = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            # Newline indicates a new sentence.
            if not line.rstrip():
                token_sents.append(token_sent)
                token_sent = []
                tag_sents.append(tag_sent)
                tag_sent = []
            else:
                token, tag = line.rstrip().split("\t")
                token_sent.append(token)
                tag_sent.append(tag)

    return token_sents, tag_sents


def make_labels2i(labels_filepath: str):
    # Initialize labels2i with the PAD_SYMBOL.
    labels2i = {PAD_SYMBOL: 0}
    i = 1
    with open(labels_filepath, "r", encoding="utf-8") as f:
        for line in f:
            if line.rstrip():
                labels2i[line.rstrip()] = i
                i += 1

    
    return labels2i

# Metrics

In [61]:
def precision(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    outside_tag_idx: int
):
    """
    Precision is True Positives / All Positives Predictions
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[pred != outside_tag_idx])
        denom += sum(pred != outside_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def recall(
    predicted_labels: List[torch.Tensor],
    true_labels: List[torch.Tensor],
    outside_tag_idx: int
):
    """
    Recall is True Positives / All Positive Labels
    """
    TP = torch.tensor([0])
    denom = torch.tensor([0])
    for pred, true in zip(predicted_labels, true_labels):
        TP += sum((pred == true)[true != outside_tag_idx])
        denom += sum(true != outside_tag_idx)

    # Avoid division by 0
    denom = torch.tensor(1) if denom == 0 else denom
    return TP / denom


def f1_score(predicted_labels, true_labels, outside_tag_idx):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, outside_tag_idx)
    R = recall(predicted_labels, true_labels, outside_tag_idx)
    return 2*P*R/(P+R)

# NER Model Class (CRF)

In [ ]:
class NERTagger(torch.nn.Module):
    """NER tagger in pytorch, 
    relying on (pytorch-crf)[https://pytorch-crf.readthedocs.io/en/stable/]
    """
    def __init__(self, features_dim: int, num_tags: int):
        """
        Args:
            num_tags (int): The number of NER tags.
            features_dim (int): Dimension of each feature vector.
                This should correspond to the number of possible features.
        """
        super().__init__()
        self.num_tags = num_tags
        self.features_dim = features_dim
        # The matrix for computing emission probabilities before
        # incorporating the label sequence in the CRFDecoder.
        # NOTE: We use nn.Embedding for its sparse implementation,
        #   but this is not actually getting embeddings in the general sense
        #   of learning representations. If you are interested in this detail, please ask!
        self.emissions_scorer = torch.nn.Embedding(features_dim, num_tags)
        # Initialize the CRF using pytorch-crf. This adds the structured prediction function 
        # on top of our scorer. Note that the scorer works essentially like a logistic regression classifier
        # But we add the CRF on top.
        self.crf_decoder = CRFDecoder(self.num_tags, batch_first=True)

    def forward(
        self, input_seq: torch.Tensor, tags: torch.Tensor, mask: torch.Tensor
    ) -> torch.Tensor:
        """Compute the log likelihood of the gold tags. This assumes
        we are in training mode.

        Args:
            input_seq (torch.Tensor): sequence_length x batch_size x num_features tensor
                of the featurized input sentence.
            tags (torch.Tensor): sequence_length x batch_size tensor of the tags.
            mask (torch.Tensor): THe mask marking which tokens are dummy pads.

        Returns:
             torch.Tensor: A batch_size x 1 tensor of the (float) sum of log_likelihoods
                for each tag.
        """
        # Get the probability of each label for each token.
        emissions = self.make_emissions(input_seq)
        # Compute the log likelihood of the gold tags 
        # given the input. We get y_i and y_{i-1} from the gold tags.
        return self.crf_decoder(emissions, tags, mask=mask, reduction="mean")

    def decode(self, input_seq: torch.Tensor) -> List[int]:
        emissions = self.make_emissions(input_seq)
        # Decode the argmax sequence of labels with viterbi
        return self.crf_decoder.decode(emissions)

    def make_emissions(self, input_seq: torch.Tensor) -> torch.Tensor:
        """Compute a probability distribution over the tags for each
        input word.

        Args:
            input_seq (torch.Tensor): batch_size x sequence_length x num_features tensor
                of the featurized input sentence.
        """
        # -> batch_size x seq_len x num_tags
        # This gives a distribution over the tags for each token
        # in the input sequence
        # We sum over the 2nd to last dim (#features) because we use embeddings as a hack
        # to get efficient sparse implementation.
        # Embedding each feature and summing is equivalent to the matrix-multiply
        # of a single layer.
        return torch.sum(self.emissions_scorer(input_seq), dim=-2)

# Training

In [ ]:
import random
from tqdm.autonotebook import tqdm

# TODO: Implement the training loop
# HINT: Build upon what we gave you for HW2.
# See cell below for how we call this training loop.

def training_loop(
    num_epochs,
    batch_size,
    train_features,
    train_labels,
    dev_features,
    dev_labels,
    optimizer,
    model,
    labels2i,
    pad_feature_idx
):
    # TODO: Zip the train features and labels
    samples = list(zip(train_features, train_labels))
    # TODO: Randomize them, while keeping them paired.
    random.shuffle(samples)
    # TODO: Build batches
    batches = []
    for i in range(0, len(samples), batch_size):
        batches.append(samples[i:i+batch_size])
    print("Training...")
    for i in range(num_epochs):
        losses = []
        for batch in tqdm(batches):
            # Here we get the features and labels, pad them,
            # and build a mask so that our model ignores PADs
            # We have abstracted the padding from you for simplicity, 
            # but please reach out if you'd like learn more.
            features, labels = zip(*batch)
            features = pad_features(features, pad_feature_idx)
            features = torch.stack(features)
            # Pad the label sequences to all be the same size, so we
            # can form a proper matrix.
            labels = pad_labels(labels, labels2i[PAD_SYMBOL])
            labels = torch.stack(labels)
            mask = (labels != labels2i[PAD_SYMBOL])
            # TODO: Empty the dynamic computation graph
            optimizer.zero_grad()
            # TODO: Run the model. Since we use the pytorch-crf model,
            # our forward function returns the positive log-likelihood already.
            # We want the negative log-likelihood. See crf.py forward method in NERTagger
            loss = -model.forward(features, labels, mask)
            # TODO: Backpropogate the loss through our model
            loss.backward()
            # TODO: Update our coefficients in the direction of the gradient.
            optimizer.step()
            # TODO: Store the losses for logging
            losses.append(loss.item())
        # TODO: Log the average Loss for the epoch
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # TODO: make dev predictions with the `predict()` function
        dev_f1 = f1_score(predict(model, dev_features), dev_labels, outside_tag_idx=labels2i['O'])
        # TODO: Compute F1 score on the dev set and log it.
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model